In [78]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
import os
import sys
from tqdm import tqdm

sys.path.append('../../ieegpy/ieeg')
sys.path.append('code/tools')
sys.path.append('code')
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# sets path to one directory up from where code is
path = "/".join(os.path.abspath(os.getcwd()).split('/')[:-1])

import json
import numpy as np
import pandas as pd
from get_iEEG_data import get_iEEG_data
from plot_iEEG_data import plot_iEEG_data
from line_length import line_length
from get_iEEG_duration import get_iEEG_duration
import csv_utils
import segmentation
import feature_extraction

[autoreload of autoencoder failed: Traceback (most recent call last):
  File "c:\users\17666\desktop\ieeg_toy_template\venv\lib\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "c:\users\17666\desktop\ieeg_toy_template\venv\lib\site-packages\IPython\extensions\autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "c:\users\17666\desktop\ieeg_toy_template\venv\lib\site-packages\IPython\extensions\autoreload.py", line 377, in update_generic
    update(a, b)
  File "c:\users\17666\desktop\ieeg_toy_template\venv\lib\site-packages\IPython\extensions\autoreload.py", line 329, in update_class
    if update_generic(old_obj, new_obj):
  File "c:\users\17666\desktop\ieeg_toy_template\venv\lib\site-packages\IPython\extensions\autoreload.py", line 377, in update_generic
    update(a, b)
  File "c:\users\17666\desktop\ieeg_toy_template\venv\lib\site-packages\IPython\extensions\autoreload.py", line 

In [3]:
with open("credentials.json") as f:
    credentials = json.load(f)
    username = credentials['username']
    password = credentials['password']

In [4]:
metadata = pd.read_csv('ieeg_metadata.csv')
metadata.describe()

,Unnamed: 0,Seizure number,Seizure EEC,Seizure UEO,Seizure end,Seizure duration
count,256.000000,256.000000,2.560000e+02,2.560000e+02,2.560000e+02,256.000000
mean,161.523438,3.808594,3.850046e+05,3.850087e+05,3.850987e+05,89.983726
std,105.468253,4.411047,2.512188e+05,2.512184e+05,2.512276e+05,107.030035
min,0.000000,0.000000,1.422600e+04,1.422600e+04,1.425995e+04,4.900000
25%,64.750000,1.000000,2.076900e+05,2.076916e+05,2.077277e+05,34.252500
50%,155.500000,2.000000,3.487700e+05,3.487708e+05,3.490025e+05,67.945000
75%,255.250000,5.000000,5.301436e+05,5.301436e+05,5.302733e+05,91.760000
max,364.000000,22.000000,1.694568e+06,1.694570e+06,1.694644e+06,781.950000


In [5]:
seizure_data = csv_utils.read_pickle('data/seizure/seizure_clips_data', 256)
seizure_time = csv_utils.read_pickle('data/seizure_time/seizure_time_data', 256)
for i in range(len(seizure_data)):
    seizure_data[i] = seizure_data[i].dropna(axis='columns')

In [6]:
nonseizure_data = csv_utils.read_pickle('data/non_seizure/nonseizure_clips_data', 256)
nonseizure_time = csv_utils.read_pickle('data/non_seizure_time/nonseizure_time_data', 256)
for i in range(len(nonseizure_data)):
    nonseizure_data[i] = nonseizure_data[i].dropna(axis='columns')

In [7]:
# seizure_data,seizure_time = csv_utils.seizure_clip_read(metadata, username, password)
# csv_utils.dump_pickle(seizure_data,'data/seizure/seizure_clips_data')
# csv_utils.dump_pickle(seizure_time,'data/seizure_time/seizure_time_data')

In [8]:
# nonseizure_data, nonseizure_time = csv_utils.nonseizure_clip_read(metadata, username, password)
# csv_utils.dump_pickle(nonseizure_data,'data/non_seizure/nonseizure_clips_data')
# csv_utils.dump_pickle(nonseizure_time,'data/non_seizure_time/nonseizure_time_data')

In [9]:
window_len = 2
stride = 1
seizure_data_windowed = []
for i in tqdm(range(len(seizure_data))):
    fs = segmentation.get_fs(seizure_time[i])
    if fs == 512:
        seg = segmentation.get_windoweds(seizure_data[i].to_numpy(), fs, window_len, stride)
        seizure_data_windowed.append(seg)
window_len = 2
stride = 1
nonseizure_data_windowed = []
for i in tqdm(range(len(nonseizure_data))):
    fs = segmentation.get_fs(nonseizure_time[i])
    if fs == 512:
        seg = segmentation.get_windoweds(nonseizure_data[i].to_numpy(), fs, window_len, stride)
        nonseizure_data_windowed.append(seg)

100%|██████████| 256/256 [00:52<00:00,  4.91it/s]


In [17]:
seizure_data_windowed[1].shape

(78, 1024, 82)

In [26]:
channel_n = 90
fs = 512
seizure_data_stack = np.empty((0, fs * 2, channel_n))
for c in seizure_data_windowed:
    if c.shape[2] == channel_n:
        seizure_data_stack = np.vstack((seizure_data_stack, c))

In [25]:
seizure_data_stack.shape

(1212, 1024, 90)

In [49]:
import keras
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D, Conv1D, MaxPooling1D, BatchNormalization,GlobalAveragePooling2D, Input, Reshape, UpSampling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.optimizers import schedules, SGD, Adam
import tensorflow as tf
from sklearn import preprocessing
from matplotlib import pyplot as plt

In [86]:
from autoencoder import Autoencoder
ae = Autoencoder(seizure_data_stack.shape[1],seizure_data_stack.shape[2])

In [93]:
ae.build(input_shape = (None,seizure_data_stack.shape[1],seizure_data_stack.shape[2]))
ae.summary()
ae.compile(optimizer='adam', loss='mae')
ae.fit(seizure_data_stack, seizure_data_stack,
          epochs=20,
          batch_size=64,
          validation_data=(seizure_data_stack, seizure_data_stack))

Model: "autoencoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_37 (Sequential)  (None, 64, 1, 64)         16128     
                                                                 
 sequential_38 (Sequential)  (None, 1024, 90)          19163     
                                                                 
Total params: 35,291
Trainable params: 35,291
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
19/19 [==============================] - 77s 4s/step - loss: 161.1964 - val_loss: 160.4370
Epoch 2/20
12/19 [=================>............] - ETA: 21s - loss: 160.5526

KeyboardInterrupt: 